<a href="https://colab.research.google.com/github/djgreen/AI-BootCamp/blob/main/AIBootCampsPandasIntro2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Wrangling with pandas - Combining Datasets: Concat, Append, Join, and Merge

**Authors**: 
- Dr. Jany Chan, The Ohio State University
- Dr. Chaitanya Kulkarni, The Ohio State University
- Prof. Raghu Machiraju, The Ohio State University

---

## Context
The material here was developed by the authors for a professional masters course in data analytics. The enrolled students are often from all academic backgrounds. MDs, PharmDs, MBAs, etc. The goal of that program is to teach to data story telling in context.

---
## Objectives 
- Learn data wrangling with pandas
- Lear differences with Numpy
- Learn how to concat data
- Learn how to combine data

---

## Data Wrangling w/ pandas objects

Now, we will discuss one aspect of data wrangling using pandas objects. Many data analyses required combining different data sources.

These operations can require straightforward concatenation of two or three different datasets to more complicated database-style joins and merges to correctly handle overlaps between datasets. 


---


Recall that pandas ``Series``s and ``DataFrame``s:
- allow fast and straightforward data wrangling 
- includes helpful, built-in functions for data wrangling


---


There are four main methods for combining different datasets:

*  `concat()` gives the flexibility to join based on the axis (all rows or all columns)

*  `append()` is the specific case of `concat(axis=0, join='outer')` and creates a new pandas object. Note: this doubles the resources utilized.

*  `join()` is based on the indexes (specified by `set_index`) on the variables `left`,`right`,`inner`, and `outer`

*  `merge()` is based on the user-defined column from each of the two dataframes based on variables like `left_on`, `right_on`, `on`


In [ ]:
# Again, let's import the packages we need
import pandas as pd
import numpy as np

In [ ]:
# And create a toy DataFrame to get us started:
def make_df(cols, ind):
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

make_df('ABCDE', range(5))

,A,B,C,D,E
0,A0,B0,C0,D0,E0
1,A1,B1,C1,D1,E1
2,A2,B2,C2,D2,E2
3,A3,B3,C3,D3,E3
4,A4,B4,C4,D4,E4


In [ ]:
# We haven't talked about displaying objects. To make comparisons easier, we'll
# create a class that allows us to display multiple DataFrame's side by side. 
# Don't worry about this code. It's to help display data within this notebook.

class display(object):
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

## Recall: Concatenation of NumPy Arrays
NumPy ndarrays can be concatenated via the ``np.concatenate()`` function, which combines the contents of two or more arrays into a single array:

In [ ]:
# Here we have 3 Python lists that we will concatenate using NumPy
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
# Note: The `axis` keyword is used here to specify the axis for concatenation:
# The default axis=0 denotes rows while axis=1 denotes columns
x = [[1, 3],
     [5, 4]]
np.concatenate([x, x], axis=1)

array([[1, 3, 1, 3],
       [5, 4, 5, 4]])

## Part 1. Simple Concatenation with ``pd.concat()``

The pandas function ``pd.concat()`` is similar in syntax to ``np.concatenate()`` with many additional options and can be used on both `DataFrames` and `Series`. The keyword `join` is borrowed from relational databases. We'll explore this situation in a [later section](#joins).

```python
# Function call in Pandas v0.18
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
```




In [ ]:
# Example - Concatentate 2 pandas Series
# Note: Objects to be concatenated are enclosed in a Python list
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[5, 6, 7])
pd.concat([ser1, ser2])

1    A
2    B
3    C
5    D
6    E
7    F
dtype: object

In [ ]:
# Example - Concatenate 2 pandas DataFrames:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])

display('df1', 'df2', 'pd.concat([df1, df2])')
# Points to ponder: 
# Can you concatenate a Series with a DataFrame?
# Is there a simpler way to achieve this result?

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


In [ ]:
# Note: By default, concatenation happens row-wise within the DataFrame 
# We can explicitly chose how we concatenate by declaring an axis: 
# (i.e., axis=0 denotes rows and axis=1 denotes columns).
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])

display('df3', 'df4', "pd.concat([df3, df4], axis=1)")

df3
    A   B
0  A0  B0
1  A1  B1

df4
    C   D
0  C0  D0
1  C1  D1

pd.concat([df3, df4], axis=1)
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1

### Duplicate indices

In [ ]:
# Note: An important difference between np.concatenate() and pd.concat() is that
# pandas' concatenation *preserves indices*, even if result has duplicate indices:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])

# Force duplicate indices
y.index = x.index   

# Now what happens when we concatenate?
display('x', 'y', 'pd.concat([x, y])')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


Notice the repeated indices in the result.
While this is valid within ``DataFrame``s, the outcome is often undesirable.
``pd.concat()`` gives us a few ways to handle it.

#### Catching the repeats as an error

To ensure that indices in the result of ``pd.concat()`` do not overlap, you can specify the ``verify_integrity`` flag. Concatenation will raise an exception if there are duplicate indices. An example below:

In [ ]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


#### Ignoring the index
Or we can ignore the index using ``ignore_index`` flag.
When this flag is set to TRUE, the concatenation will create a new integer index for the resulting ``Series``:

In [ ]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


#### Adding Multi-Index keys

 Alternatively, we can use ``keys`` to specify a label for data sources; the result is a hierarchically indexed series containing data:

In [ ]:
display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")

x
    A   B
0  A0  B0
1  A1  B1

y
    A   B
0  A2  B2
1  A3  B3

pd.concat([x, y], keys=['x', 'y'])
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

### Concatenation with joins <a name="joins"></a>

So far  we only concatenated ``DataFrame``s with shared column names. 
 Data from different sources will have different sets of column names, and ``pd.concat()`` offers several options. 

In [ ]:
# Let's consider 2 DataFrames that have a few columns in common (B and C):
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 5])

display('df5', 'df6', 'pd.concat([df5, df6])')

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
5  B5  C5  D5

pd.concat([df5, df6])
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
5  NaN  B5  C5   D5

By default, `NaN` is used in case of missing values. A better alternative is to specify options for the ``join`` parameter of the concatenate function. Similar to joins in a realtional database, the pandas join is a union of the input columns (``join='outer'``) or intersection of the columns using ``join='inner'``. 

Here's a quick visualization of the common joins between 2 different sets:
![picture](https://drive.google.com/uc?id=1hsFNrtR-RryoenNuWjt7FmU7puUyeKz0)

In [ ]:
display('df5', 'df6', "pd.concat([df5, df6], join='inner')")

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
5  B5  C5  D5

pd.concat([df5, df6], join='inner')
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
5  B5  C5

### Using ``append()``
We can use the ``append`` method to combine pandas DataFrames. However, ``append()``  creates a new object with a new index at another memory location (i.e. data buffer).  Thus, it is not a very efficient method when dealing with large datasets unless we deliberately want a new object to be created.

In [ ]:
display('df1', 'df2', 'df1.append(df2)')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4



In terms of computational efficiency, `append()` < `concat()` < `merge()` 

Let's take a look at pandas `merge()` in the next section.


## Part 2. Combining Datasets: Merge and Join

One of the defining features of pandas is its high-performance, in-memory join and merge operations. *This is the main reason pandas is used for data analysis*. These operations have been derived from methods used to organize and use databases. The workhorse is the ``pd.merge()`` function.

### Relational Algebra

 ``pd.merge()`` implements a subset of *relational algebra*, a formal set of rules for manipulating relational data. Relational algebra is the underpining of modern databases. Several primitive operations become building blocks of more complicated and composite operations on any dataset.

#### Categories of Joins

The ``pd.merge()`` function implements the following three joins: 
*  *one-to-one*
*  *many-to-one*
* *many-to-many*

You can visualize these joins in the following image where data is being merged from the left set to the right or vice versa:
![picture](https://drive.google.com/uc?id=11S0IfAzj5FbQhulBTldyD42pr4PV21IT)

#### One-to-one joins

These joins are similar to column-wise concatenation. 

In [ ]:
# Consider these 2 DataFrames that contain information on employees in a company:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})

df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})

display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


The default behavior of ``pd.merge()`` is to search for one or more matching column names in the inputs to use as the key.  

Here, ``pd.merge()``:
1. identifies the "employee" column as a valid key,
2. joins both ``DataFrame``s using this column, and
3. creates  a new ``DataFrame``.

Even if the order of entries in each column is not maintained, the ``pd.merge()`` function correctly accounts for this. Also note that `merge()` in general discards the original index.

In [ ]:
# To combine datasets into a single DataFrame with pd.merge():
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


### Many-to-one joins

Many-to-one joins are those in which one of the two key columns contains duplicate entries. The resulting ``DataFrame`` will preserve those duplicate entries as appropriate.

Note the additional column with the "supervisor" information in final ``DataFrame``. 

In [ ]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})

display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

### Many-to-many joins

Consider a ``DataFrame`` showing one or more skills associated with a particular group. 

In the following example, a many-to-many join will display all skills for all individuals:

In [ ]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})

display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


## Specification of the Merge Key

Recall that the default behavior of ``pd.merge()`` is to look for one or more matching column names in the inputs for use as the key. However, mismatches, typos, or different naming conventions across datasets can complicate this merge. 

In these cases, ``pd.merge()`` provides addtional, optional keywords:

### The ``on`` keyword

Just specify the name of the key column using the ``on`` keyword, which takes a column or list of columns. 

Note: This method only works if both the left and right ``DataFrame``s have the specified column name.

In [ ]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

### The ``left_on`` and ``right_on`` keywords

What about if the two datasets have different column names?  

In our running examples, let the employees' names be labeled as "name" instead of "employee". 

We can use ``left_on`` and ``right_on`` keywords to resolve the conflct:

In [ ]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})

display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

However, now we have a redundant information; the "employee" and "name" columns are effectively duplicates. 

Use the ``drop()`` function to remove one.

Note: As mentioned in earlier notebooks, we can chain function calls one after another. Here, first we call `merge` on our datasets and then the `drop` function is applied to the result. **Chaining is evaluated sequentially from left to right.**

In [ ]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### The ``left_index`` and ``right_index`` keywords

In [ ]:
# How about merging on an index?
# First, let's set the index as 'employee'
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')

display('df1a', 'df2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


You can use the index as the key for merging by specifying the ``left_index`` and/or ``right_index`` flags in ``pd.merge()``:

In [ ]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Finally, indices and columns can be mixed by combining ``left_index`` with ``right_on`` or ``left_on`` with ``right_index``:

In [ ]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')")

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1a, df3, left_index=True, right_on='name')
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000

## Using ``join()``

The ``join()`` method performs a merge that defaults to joining on indices:

In [ ]:
display('df1a', 'df2a', 'df1a.join(df2a)')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


## Specifying Set Arithmetic for Joins

So far we ignored the type of set arithmetic used in the join.

This is important when  a value appears in one key column but not the other:

In [ ]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])

df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])

display('df6', 'df7', 'pd.merge(df6, df7)')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

Only a single "name" entry is shared between the two datasets, namely Mary. 

Here, `merge()` functions similarly to `concat()`. The result is the *intersection* of the two sets of inputs or an *inner join*. The ``how`` keyword (default=``"inner"``) can specify from options including ``'outer'``, ``'left'``, and ``'right'``.:

Recall this image on joins from above:
![picture](https://drive.google.com/uc?id=1hsFNrtR-RryoenNuWjt7FmU7puUyeKz0)

In [ ]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine



The *outer join* returns a join over the union of input columns, and fills all missing values with NaNs:

In [ ]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='outer')
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer

The *left join* and *right join* return joins over the left entries and right entries, respectively.
For example:

In [ ]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

Output rows now correspond to the entries in the left input. Similar output for
``how='right'``.

## Overlapping Column Names: The ``suffixes`` Keyword

Earlier, we covered how to resolve columns of the same data but with mismatched names. 

What about columns that have the same name but contain different data?:

In [ ]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})

df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})

display('df8', 'df9', 'pd.merge(df8, df9, on="name")')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

The `merge()` function automatically appends a suffix ``_x`` or ``_y`` to make the output columns unique.  Or, we can specify a custom suffix using ``suffixes`` keyword. Suffixes work in any  possible join patterns and if there are also multiple overlapping columns.

In [ ]:
display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2